In [ ]:
# libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from math import sqrt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ENSO_data = pd.read_csv("/content/drive/MyDrive/MCM Practicum/ENSO.csv")

In [ ]:
ENSO_data.describe()

,Year,PNA,SOI
count,859.000000,859.000000,859.000000
mean,1986.293364,-0.094820,0.145402
std,20.677287,1.030522,0.938163
min,1951.000000,-3.190000,-3.600000
25%,1968.000000,-0.760000,-0.400000
50%,1986.000000,-0.070000,0.200000
75%,2004.000000,0.605000,0.800000
max,2022.000000,2.870000,2.900000


# Enso Indicator  

> If SOI > 0.5 -> Warm -> El Nino

> If SOI < 0.5 -> Cool -> La Nina





In [ ]:
ENSO_data.head()

,Date,Year,Month,Season,ONI,NINO 1+2 SST,NINO 1+2 SST Anomalies,NINO 3 SST,NINO 3 SST Anomalies,NINO 3.4 SST,NINO 3.4 SST Anomalies,NINO 4 SST,NINO 4 SST Anomalies,OLR,TNI,PNA,Precipitation (mm/day),SOI,Effects
0,01-01-1951,1951,JAN,DJF 1951,-0.8,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,1.315,-1.18,NAN,1.5,El Nino
1,01-02-1951,1951,FEB,JFM 1951,-0.5,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,1.461,-2.11,NAN,0.9,El Nino
2,01-03-1951,1951,MAR,FMA 1951,-0.2,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,1.59,-1.09,NAN,-0.1,La Nina
3,01-04-1951,1951,APR,MAM 1951,0.2,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,1.457,0.47,NAN,-0.3,La Nina
4,01-05-1951,1951,MAY,AMJ 1951,0.4,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,NAN,1.615,1.19,NAN,-0.7,La Nina


In [ ]:
ENSO_data['Date'] = pd.to_datetime(ENSO_data['Date'])

In [ ]:
start_date = pd.to_datetime('01-01-1982')
end_date = pd.to_datetime('01-12-2021')

data = ENSO_data[(ENSO_data['Date']>= start_date) & (ENSO_data['Date']<= end_date)]
data.head()

,Date,Year,Month,Season,ONI,NINO 1+2 SST,NINO 1+2 SST Anomalies,NINO 3 SST,NINO 3 SST Anomalies,NINO 3.4 SST,NINO 3.4 SST Anomalies,NINO 4 SST,NINO 4 SST Anomalies,OLR,TNI,PNA,Precipitation (mm/day),SOI,Effects,Effect
372,1982-01-01,1982,JAN,DJF 1982,-0.1,24.28,-0.24,25.84,0.17,26.65,0.08,28.01,-0.21,0.9,-1.067,-1.75,5.66,1.2,El Nino,El-Nino
373,1982-01-02,1982,FEB,JFM 1982,0.1,25.38,-0.72,26.26,-0.11,26.54,-0.2,27.99,-0.11,0.1,-1.466,-1.20,5.5,0.3,La Nina,La-Nina
374,1982-01-03,1982,MAR,FMA 1982,0.2,25.22,-1.38,26.92,-0.25,27.09,-0.14,28.18,-0.05,-0.6,-1.922,-1.66,0.44,0.6,El Nino,El-Nino
375,1982-01-04,1982,APR,MAM 1982,0.5,24.57,-1.16,27.52,-0.05,27.83,0.02,28.61,0.1,0,-2.359,-2.08,0.28,0.1,La Nina,La-Nina
376,1982-01-05,1982,MAY,AMJ 1982,0.7,24,-0.62,27.7,0.49,28.37,0.49,29.19,0.4,0.4,-2.364,-0.53,2.51,-0.3,La Nina,La-Nina


In [ ]:
data = data.drop(['Effect'], axis=1)

In [ ]:
data.head()

,Date,Year,Month,Season,ONI,NINO 1+2 SST,NINO 1+2 SST Anomalies,NINO 3 SST,NINO 3 SST Anomalies,NINO 3.4 SST,NINO 3.4 SST Anomalies,NINO 4 SST,NINO 4 SST Anomalies,OLR,TNI,PNA,Precipitation (mm/day),SOI,Effects
372,1982-01-01,1982,JAN,DJF 1982,-0.1,24.28,-0.24,25.84,0.17,26.65,0.08,28.01,-0.21,0.9,-1.067,-1.75,5.66,1.2,El Nino
373,1982-01-02,1982,FEB,JFM 1982,0.1,25.38,-0.72,26.26,-0.11,26.54,-0.2,27.99,-0.11,0.1,-1.466,-1.20,5.5,0.3,La Nina
374,1982-01-03,1982,MAR,FMA 1982,0.2,25.22,-1.38,26.92,-0.25,27.09,-0.14,28.18,-0.05,-0.6,-1.922,-1.66,0.44,0.6,El Nino
375,1982-01-04,1982,APR,MAM 1982,0.5,24.57,-1.16,27.52,-0.05,27.83,0.02,28.61,0.1,0,-2.359,-2.08,0.28,0.1,La Nina
376,1982-01-05,1982,MAY,AMJ 1982,0.7,24,-0.62,27.7,0.49,28.37,0.49,29.19,0.4,0.4,-2.364,-0.53,2.51,-0.3,La Nina


###Calculating SPI

In [ ]:
d_data = pd.read_csv('/content/drive/MyDrive/MCM Practicum/Drought_1.csv')

In [ ]:
d_data.head()

,YEAR,DOY,T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN,ALLSKY_SFC_LW_DWN,QV2M,RH2M,PRECTOTCORR,GWETTOP,GWETROOT,GWETPROF
0,2000,1,20.13,1.11,10.62,21.09,18.48,30.14,11.67,335.5,4.39,31.56,0.0,0.23,0.59,0.6
1,2000,2,20.03,0.70,10.37,20.99,17.95,29.82,11.87,329.9,4.21,30.69,0.0,0.23,0.59,0.6
2,2000,3,19.96,-0.36,9.80,20.97,17.87,29.87,12.00,329.9,3.91,28.44,0.0,0.23,0.59,0.6
3,2000,4,19.80,-2.26,8.76,20.80,18.05,29.90,11.85,326.3,3.42,25.25,0.0,0.23,0.59,0.6
4,2000,5,19.30,-2.79,8.25,20.48,19.01,29.58,10.58,322.9,3.23,25.06,0.0,0.22,0.59,0.6


In [ ]:
d_data['Date'] = pd.to_datetime(d_data['YEAR'], format='%Y') + pd.to_timedelta(d_data['DOY'] - 1, unit='D')

In [ ]:
!pip install standard-precip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from standard_precip.spi import SPI
from standard_precip.utils import plot_index

In [ ]:
spi = SPI()
new_spi = SPI()

In [ ]:
d_data

,YEAR,DOY,T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN,ALLSKY_SFC_LW_DWN,QV2M,RH2M,PRECTOTCORR,GWETTOP,GWETROOT,GWETPROF,Date
0,2000,1,20.13,1.11,10.62,21.09,18.48,30.14,11.67,335.50,4.39,31.56,0.00,0.23,0.59,0.60,2000-01-01
1,2000,2,20.03,0.70,10.37,20.99,17.95,29.82,11.87,329.90,4.21,30.69,0.00,0.23,0.59,0.60,2000-01-02
2,2000,3,19.96,-0.36,9.80,20.97,17.87,29.87,12.00,329.90,3.91,28.44,0.00,0.23,0.59,0.60,2000-01-03
3,2000,4,19.80,-2.26,8.76,20.80,18.05,29.90,11.85,326.30,3.42,25.25,0.00,0.23,0.59,0.60,2000-01-04
4,2000,5,19.30,-2.79,8.25,20.48,19.01,29.58,10.58,322.90,3.23,25.06,0.00,0.22,0.59,0.60,2000-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7667,2020,363,19.61,12.14,15.87,19.18,14.47,27.26,12.79,352.03,9.34,64.50,0.00,0.66,0.66,0.68,2020-12-28
7668,2020,364,19.08,12.08,15.58,19.12,15.62,27.71,12.08,349.76,9.34,67.00,0.00,0.65,0.66,0.67,2020-12-29
7669,2020,365,19.12,12.74,15.93,19.11,15.30,27.55,12.25,352.66,9.70,69.75,0.00,0.65,0.66,0.67,2020-12-30
7670,2020,366,18.86,12.46,15.65,18.96,13.95,26.92,12.97,364.25,9.52,69.44,0.00,0.65,0.66,0.67,2020-12-31


In [ ]:
d_data

,YEAR,DOY,T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN,ALLSKY_SFC_LW_DWN,QV2M,RH2M,PRECTOTCORR,GWETTOP,GWETROOT,GWETPROF,Date
0,2000,1,20.13,1.11,10.62,21.09,18.48,30.14,11.67,335.50,4.39,31.56,0.00,0.23,0.59,0.60,2000-01-01
1,2000,2,20.03,0.70,10.37,20.99,17.95,29.82,11.87,329.90,4.21,30.69,0.00,0.23,0.59,0.60,2000-01-02
2,2000,3,19.96,-0.36,9.80,20.97,17.87,29.87,12.00,329.90,3.91,28.44,0.00,0.23,0.59,0.60,2000-01-03
3,2000,4,19.80,-2.26,8.76,20.80,18.05,29.90,11.85,326.30,3.42,25.25,0.00,0.23,0.59,0.60,2000-01-04
4,2000,5,19.30,-2.79,8.25,20.48,19.01,29.58,10.58,322.90,3.23,25.06,0.00,0.22,0.59,0.60,2000-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7667,2020,363,19.61,12.14,15.87,19.18,14.47,27.26,12.79,352.03,9.34,64.50,0.00,0.66,0.66,0.68,2020-12-28
7668,2020,364,19.08,12.08,15.58,19.12,15.62,27.71,12.08,349.76,9.34,67.00,0.00,0.65,0.66,0.67,2020-12-29
7669,2020,365,19.12,12.74,15.93,19.11,15.30,27.55,12.25,352.66,9.70,69.75,0.00,0.65,0.66,0.67,2020-12-30
7670,2020,366,18.86,12.46,15.65,18.96,13.95,26.92,12.97,364.25,9.52,69.44,0.00,0.65,0.66,0.67,2020-12-31


In [ ]:
df_spi = new_spi.calculate(
    d_data,
    'Date',
    'PRECTOTCORR',
    freq="M",
    scale=1,
    fit_type="lmom",
    dist_type="gam"
)

In [ ]:
df_spi

,Date,PRECTOTCORR,PRECTOTCORR_calculated_index
0,2000-01-01,0.00,1.292110
1,2000-01-02,0.00,1.292110
2,2000-01-03,0.00,1.292110
3,2000-01-04,0.00,1.292110
4,2000-01-05,0.00,1.292110
...,...,...,...
7667,2020-12-28,0.00,1.482344
7668,2020-12-29,0.00,1.482344
7669,2020-12-30,0.00,1.482344
7670,2020-12-31,0.00,1.482344


In [ ]:
df_spi['PRECTOTCORR_calculated_index']

0       1.292110
1       1.292110
2       1.292110
3       1.292110
4       1.292110
          ...   
7667    1.482344
7668    1.482344
7669    1.482344
7670    1.482344
7671    1.490562
Name: PRECTOTCORR_calculated_index, Length: 7672, dtype: float64

In [ ]:
d_data['SPI_Index'] = df_spi['PRECTOTCORR_calculated_index']

In [ ]:
d_data

,YEAR,DOY,T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN,ALLSKY_SFC_LW_DWN,QV2M,RH2M,PRECTOTCORR,GWETTOP,GWETROOT,GWETPROF,Date,SPI_Index
0,2000,1,20.13,1.11,10.62,21.09,18.48,30.14,11.67,335.50,4.39,31.56,0.00,0.23,0.59,0.60,2000-01-01,1.292110
1,2000,2,20.03,0.70,10.37,20.99,17.95,29.82,11.87,329.90,4.21,30.69,0.00,0.23,0.59,0.60,2000-01-02,1.292110
2,2000,3,19.96,-0.36,9.80,20.97,17.87,29.87,12.00,329.90,3.91,28.44,0.00,0.23,0.59,0.60,2000-01-03,1.292110
3,2000,4,19.80,-2.26,8.76,20.80,18.05,29.90,11.85,326.30,3.42,25.25,0.00,0.23,0.59,0.60,2000-01-04,1.292110
4,2000,5,19.30,-2.79,8.25,20.48,19.01,29.58,10.58,322.90,3.23,25.06,0.00,0.22,0.59,0.60,2000-01-05,1.292110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7667,2020,363,19.61,12.14,15.87,19.18,14.47,27.26,12.79,352.03,9.34,64.50,0.00,0.66,0.66,0.68,2020-12-28,1.482344
7668,2020,364,19.08,12.08,15.58,19.12,15.62,27.71,12.08,349.76,9.34,67.00,0.00,0.65,0.66,0.67,2020-12-29,1.482344
7669,2020,365,19.12,12.74,15.93,19.11,15.30,27.55,12.25,352.66,9.70,69.75,0.00,0.65,0.66,0.67,2020-12-30,1.482344
7670,2020,366,18.86,12.46,15.65,18.96,13.95,26.92,12.97,364.25,9.52,69.44,0.00,0.65,0.66,0.67,2020-12-31,1.482344


In [3]:
#!pip install spei

In [1]:
#import spei as si

In [2]:
#import scipy.stats as scs
#spei = si.spei(d_data['PRECTOTCORR'])